# Exercise #3: Exploring the phase diagram of NaCl

You now know how to set up and run MD simulations, and analyze their outputs. Still using the same system (NaCl crystal in a 4×4×4 supercell) this is now a lot more open notebook for you to explore and answer some questions with your own code, analysis and reasoning.

In [1]:
%%bash
[[ -z "$COLAB_RELEASE_TAG" ]] && exit
# Install required packages if running on Google Colab
pip install mdanalysis nglview openmm tidynamics

In [2]:
from io import StringIO
import tempfile
import warnings

import numpy as np
import pandas as pd

import MDAnalysis as mda
import MDAnalysis.analysis.rdf, MDAnalysis.analysis.msd
import nglview

import openmm
import openmm.app
import openmm.unit

try:
    from google.colab import output
    output.enable_custom_widget_manager()
    IN_COLAB = True
except:
    IN_COLAB = False

## Wrapping the MD simulation inside a function

We will use the same function as previously, to perform the MD itself.

In [3]:
offxml = """
<ForceField>
<AtomTypes>
 <Type name="0" class="Na" element="Na" mass="22.990"/>
 <Type name="1" class="Cl" element="Cl" mass="35.453"/>
 ...
</AtomTypes>

<Residues>
 <Residue name="Na">
  <Atom name="Na" type="0"/>
 </Residue>
 <Residue name="Cl">
  <Atom name="Cl" type="1"/>
 </Residue>
</Residues>

<NonbondedForce coulomb14scale="0.0" lj14scale="0.0">
 <Atom type="0" charge="1.0" sigma="0.3330" epsilon="0.011598"/>
 <Atom type="1" charge="-1.0" sigma="0.4417" epsilon="0.493712"/>
</NonbondedForce>
</ForceField>
"""

forcefield = openmm.app.forcefield.ForceField(StringIO(offxml))

In [4]:
class ExtendedStateDataReporter(openmm.app.StateDataReporter):
    """
    An extension of OpenMM's StateDataReporter_ class, which also outputs pressure if a barostat is present
    """
    # Inspired by https://atomsmm.readthedocs.io/en/latest/_modules/atomsmm/reporters.html#ExtendedStateDataReporter
    def __init__(self, file, reportInterval, **kwargs):
        self._barostat = kwargs.pop('barostat', None)
        super().__init__(file, reportInterval, **kwargs)

    def _constructHeaders(self):
        headers = super()._constructHeaders()
        if self._barostat:
            headers.append('Pressure (atm)')
        return headers

    def _constructReportValues(self, simulation, state):
        values = super()._constructReportValues(simulation, state)
        if self._barostat:
            pressure = self._barostat.computeCurrentPressure(simulation.context)
            values.append(pressure.value_in_unit(openmm.unit.atmospheres))
        return values

Below is the most important function, `perform_nacl_md`, which is responsible for all the work. It will set up the system, force field, and MD integration, perform the molecular dynamics, and load the resulting data into Python objects.

It has several parameters:
- `temperature` of our $(N, P, T)$ ensemble.
- `pressure` of our $(N, P, T)$ ensemble.
- `nsteps`: number of MD steps to perform.
- `timestep`: ∆t, the length (in units of time) of one step of MD. Default value: 1 fs.
- `friction`: thermostat parameter, strength of the coupling with the heat bath. Default value: 1 ps<sup>–1</sup>.
- `report_frequency`: . Default value: 50.

In [5]:
def perform_nacl_md(temperature, pressure, nsteps,
                    timestep = 0.001 * openmm.unit.picoseconds,
                    friction = 1. / openmm.unit.picoseconds,
                    report_frequency = 50):
    """
    Perform a MD simulation of NaCl 4x4x4 supercell in the NPT ensemble,
    at specific temperature and pressure. Return the data and trajectory.
    """

    # Read the starting configuration
    pdb = openmm.app.pdbfile.PDBFile('NaCl_4x4x4.pdb')

    # Check that the force field covers all atom types in the input structure
    unmatched_residues = forcefield.getUnmatchedResidues(pdb.topology)
    assert len(unmatched_residues) == 0

    # Create the system and add a barostat
    system = forcefield.createSystem(pdb.topology, nonbondedMethod=openmm.app.CutoffPeriodic, nonbondedCutoff=1.1)
    barostat = openmm.MonteCarloBarostat(pressure, temperature)
    system.addForce(barostat)

    # Specify how the integration should be performed
    integrator = openmm.LangevinMiddleIntegrator(temperature, friction, timestep)
    simulation = openmm.app.Simulation(pdb.topology, system, integrator)

    # Set up the initial state: positions and velocities
    simulation.context.setPositions(pdb.positions)
    simulation.minimizeEnergy()
    simulation.context.setVelocitiesToTemperature(temperature)

    # Output data to temporary files
    with tempfile.TemporaryDirectory() as tmpdir:
        # Write trajectory
        dcd_reporter = openmm.app.DCDReporter(f'{tmpdir}/md_trajectory.dcd', report_frequency, enforcePeriodicBox=False)
        simulation.reporters.append(dcd_reporter)

        # Output numerical data
        data_reporter = ExtendedStateDataReporter(f'{tmpdir}/md_data.csv', report_frequency,
            step=True, time=True, potentialEnergy=True, kineticEnergy=True,
            volume=True, temperature=True, density=True, barostat=barostat,
        )
        simulation.reporters.append(data_reporter)

        # Run the simulation
        simulation.step(nsteps)

        # Gather the numerical data
        data = pd.read_csv(f'{tmpdir}/md_data.csv')

        # Read the trajectory
        with warnings.catch_warnings():
            # Ignore "DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace"
            warnings.filterwarnings("ignore", category=DeprecationWarning)
            traj = mda.Universe('NaCl_4x4x4.pdb', f'{tmpdir}/md_trajectory.dcd')
            # Trajectory file will soon be removed, so we must force the trajectory in memory
            traj.transfer_to_memory()

        # Return data and trajectory
        return data, traj

What are the values returned by the `perform_nacl_md` function? There are two:
- the numerical data as a [`pandas.DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)
- the trajectory as an [`madanalysis.Universe`](https://userguide.mdanalysis.org/stable/universe.html)

We will see below how it can be exploited.

## Exercise 1: the melting temperature of NaCl

- So we've seen before that increasing the temperature leads to severe changes in the structure and dynamics of the NaCl crystal. But when exactly do you think we can say it's transitioned from the solid to the liquid state? Propose answers based on the RDF and the MSD. Determine the melting temperature of NaCl in our model, and compare it to the experimental value.
- Can you predict how pressure influences the melting temperature? And verify your prediction numerically?
- Finally, how is melting affected by the choice of thermodynamic ensemble (i.e., the simulation conditions): $(N, P, T)$ vs. $(N, V, T)$?

## Exercise 2: Thermodynamics of melting

Previously, we have used structural and dynamical criteria to 

- The proper definition of a thermodynamic phase transition (of the first order, as the solid–liquid transition) involves a discontinuity in the $U(T)$ curve of internal energy as a function of temperature. Can you plot a $U(T)$ curve and find out the proper (thermodynamic) melting temperature of your system?
- It is also possible to study the evolution of the heat capacity of the system, $C_p(T)$, as a function of temperature. $C_p$ should have a peak (maximum value) at the phase transition. You can calculate $C_p$ from the fluctuation–dissipation theorem:
  $$ \langle H^2\rangle - \langle H\rangle^2 = k_B T^2 C_p $$
  Use this to plot $C_p(T)$ and confirm the phase transition.

## Exercise 3: Quenching the liquid

You now know how to generate a configuration of molten NaCl by heating it. But what happens if you cool it down?
- Try cooling it down to different temperatures, and see what happens. What phase is formed?
- And, a bit more complicated: can you try quenching the molten NaCl at different cooling rates? (measured in K/ps) Does that affect the nature of the resulting solid?

## Exercise 4: Comparing the glass and the crystal

- How do the properties of that amorphous phase compare to that of the liquid and crystal? The structure, the dynamics, the energetics?
- If you heat the amorphous phase again, does it melt? Does it melt at the same temperature as the crystal? Does it become the “same” liquid as that obtained from heating a crystal?